# 1. Cargar Datos desde un Bucket de S3 (CSV a PySpark DataFrame)


In [1]:
from pyspark.sql import SparkSession

# Crear la sesión de Spark
spark = SparkSession.builder.appName("CO2 Emissions from S3").getOrCreate()

# Cargar el archivo CSV desde el bucket de S3
df = spark.read.csv("s3://tduquegtrabajo1-almac/raw/country/CO2 emission by countries.csv", header=True, inferSchema=True)

# Mostrar las primeras filas
df.show(5)

# Mostrar el esquema
df.printSchema()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1725855336873_0001,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+----+------------+----+-------------------+----------------+------+----------+------------+
|    Country|Code|Calling Code|Year|CO2 emission (Tons)|Population(2022)|  Area|% of World|Density(km2)|
+-----------+----+------------+----+-------------------+----------------+------+----------+------------+
|Afghanistan|  AF|          93|1750|                0.0|        41128771|652230|     0.40%|      63/km�|
|Afghanistan|  AF|          93|1751|                0.0|        41128771|652230|     0.40%|      63/km�|
|Afghanistan|  AF|          93|1752|                0.0|        41128771|652230|     0.40%|      63/km�|
|Afghanistan|  AF|          93|1753|                0.0|        41128771|652230|     0.40%|      63/km�|
|Afghanistan|  AF|          93|1754|                0.0|        41128771|652230|     0.40%|      63/km�|
+-----------+----+------------+----+-------------------+----------------+------+----------+------------+
only showing top 5 rows

root
 |-- Country: string (nul

# 2. Renombrar Columnas para un Mejor Manejo

In [10]:
# Renombrar las columnas
df = df.withColumnRenamed("Country", "country_name") \
       .withColumnRenamed("Year", "year") \
       .withColumnRenamed("CO2 emission (Tons)", "co2_emission_kt")

# Mostrar las primeras filas con las columnas renombradas
df.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+----+------------+----+---------------+----------------+------+----------+------------+
|country_name|Code|Calling Code|year|co2_emission_kt|Population(2022)|  Area|% of World|Density(km2)|
+------------+----+------------+----+---------------+----------------+------+----------+------------+
| Afghanistan|  AF|          93|1750|            0.0|        41128771|652230|     0.40%|      63/km�|
| Afghanistan|  AF|          93|1751|            0.0|        41128771|652230|     0.40%|      63/km�|
| Afghanistan|  AF|          93|1752|            0.0|        41128771|652230|     0.40%|      63/km�|
| Afghanistan|  AF|          93|1753|            0.0|        41128771|652230|     0.40%|      63/km�|
| Afghanistan|  AF|          93|1754|            0.0|        41128771|652230|     0.40%|      63/km�|
+------------+----+------------+----+---------------+----------------+------+----------+------------+
only showing top 5 rows

# 3. Análisis Básico: Emisiones Promedio de CO2 por País

In [11]:
# Calcular las emisiones promedio de CO2 por país
avg_co2_by_country = df.groupBy("country_name").avg("co2_emission_kt")

# Mostrar los resultados
avg_co2_by_country.show(10)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+--------------------+
|country_name|avg(co2_emission_kt)|
+------------+--------------------+
|      Guyana| 1.081353288191882E7|
|      Turkey| 8.121958971734318E8|
|Saint Helena|   25478.28782287823|
|   Argentina| 9.712873376715868E8|
|      Angola| 4.218294382287823E7|
|     Albania|  3.41337476199262E7|
|   Nicaragua|1.5111136811808119E7|
|        Peru| 2.220289759889299E8|
|       China|1.536124098578966...|
|     Somalia|    3294044.63099631|
+------------+--------------------+
only showing top 10 rows

# 4. Ordenar los Países por Emisiones Promedio de CO2 (De Mayor a Menor)

In [12]:
# Ordenar los países por emisiones promedio de CO2
avg_co2_by_country_sorted = avg_co2_by_country.orderBy("avg(co2_emission_kt)", ascending=False)

# Mostrar los 10 primeros países con mayores emisiones promedio
avg_co2_by_country_sorted.show(10)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------+--------------------+
|  country_name|avg(co2_emission_kt)|
+--------------+--------------------+
| United States|7.036445552702214E10|
|United Kingdom|2.273830949637269...|
|       Germany|1.955915446687085E10|
|         China|1.536124098578966...|
|        Russia|1.505807632926568...|
|        France| 8.259214850221402E9|
|         Japan| 7.568503080221402E9|
|        Poland| 4.839681718859778E9|
|        Canada| 4.632883807450185E9|
|       Ukraine| 4.477252391553506E9|
+--------------+--------------------+
only showing top 10 rows

# 5. Filtrar Datos por Año (1990 - 2020)

In [13]:
# Filtrar los datos por un rango de años
df_filtered = df.filter((df.year >= 1990) & (df.year <= 2020))

# Mostrar los datos filtrados
df_filtered.show(10)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+----+------------+----+---------------+----------------+------+----------+------------+
|country_name|Code|Calling Code|year|co2_emission_kt|Population(2022)|  Area|% of World|Density(km2)|
+------------+----+------------+----+---------------+----------------+------+----------+------------+
| Afghanistan|  AF|          93|1990|    5.9182404E7|        41128771|652230|     0.40%|      63/km�|
| Afghanistan|  AF|          93|1991|     6.160977E7|        41128771|652230|     0.40%|      63/km�|
| Afghanistan|  AF|          93|1992|    6.2989232E7|        41128771|652230|     0.40%|      63/km�|
| Afghanistan|  AF|          93|1993|    6.4322287E7|        41128771|652230|     0.40%|      63/km�|
| Afghanistan|  AF|          93|1994|    6.5603889E7|        41128771|652230|     0.40%|      63/km�|
| Afghanistan|  AF|          93|1995|    6.6834352E7|        41128771|652230|     0.40%|      63/km�|
| Afghanistan|  AF|          93|1996|    6.7999272E7|        41128771|652230|     

# 6. Análisis de Emisiones de CO2 en el Año 2020

In [14]:
# Filtrar los datos para el año 2020
df_2020 = df.filter(df.year == 2020)

# Mostrar las primeras filas del año 2020
df_2020.show(10)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+----+------------+----+---------------+----------------+-------+----------+------------+
|       country_name|Code|Calling Code|year|co2_emission_kt|Population(2022)|   Area|% of World|Density(km2)|
+-------------------+----+------------+----+---------------+----------------+-------+----------+------------+
|        Afghanistan|  AF|          93|2020|   1.92848747E8|        41128771| 652230|     0.40%|      63/km�|
|            Albania|  AL|         355|2020|   2.85502507E8|         2842321|  28748|     0.00%|      99/km�|
|            Algeria|  DZ|         213|2020|  4.587131635E9|        44903225|2381741|     1.60%|      19/km�|
|            Andorra|  AD|         376|2020|    1.5070878E7|           79824|    468|     0.00%|     171/km�|
|             Angola|  AO|         244|2020|   6.82260326E8|        35588987|1246700|     0.80%|      29/km�|
|           Anguilla|  AI|       1-264|2020|      3421269.0|            null|     91|     0.00%|        null|
|         

# 7. Encontrar el País con las Mayores Emisiones en 2020

In [15]:

df_2020.orderBy("co2_emission_kt", ascending=False).show(1)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+----+------------+----+---------------+----------------+-------+----------+------------+
| country_name|Code|Calling Code|year|co2_emission_kt|Population(2022)|   Area|% of World|Density(km2)|
+-------------+----+------------+----+---------------+----------------+-------+----------+------------+
|United States|  US|           1|2020|        4.17E11|       338289857|9372610|     6.10%|      36/km�|
+-------------+----+------------+----+---------------+----------------+-------+----------+------------+
only showing top 1 row

# 8. Agregar una Columna para Emisiones de CO2 en Millones de Toneladas

In [18]:
# Crear una nueva columna que convierta las emisiones de kilotoneladas a millones de toneladas
df = df.withColumn("co2_emission_mt", df.co2_emission_kt / 1000)

# Mostrar las primeras filas con la nueva columna
df.show(5)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+----+------------+----+---------------+----------------+------+----------+------------+---------------+
|country_name|Code|Calling Code|year|co2_emission_kt|Population(2022)|  Area|% of World|Density(km2)|co2_emission_mt|
+------------+----+------------+----+---------------+----------------+------+----------+------------+---------------+
| Afghanistan|  AF|          93|1750|            0.0|        41128771|652230|     0.40%|      63/km�|            0.0|
| Afghanistan|  AF|          93|1751|            0.0|        41128771|652230|     0.40%|      63/km�|            0.0|
| Afghanistan|  AF|          93|1752|            0.0|        41128771|652230|     0.40%|      63/km�|            0.0|
| Afghanistan|  AF|          93|1753|            0.0|        41128771|652230|     0.40%|      63/km�|            0.0|
| Afghanistan|  AF|          93|1754|            0.0|        41128771|652230|     0.40%|      63/km�|            0.0|
+------------+----+------------+----+---------------+---

# 9. Calcular el Total de Emisiones de CO2 por País Entre 1990 y 2020

In [17]:
# Filtrar los datos entre 1990 y 2020
df_filtered = df.filter((df.year >= 1990) & (df.year <= 2020))

# Calcular el total de emisiones por país en ese rango de años
total_co2_by_country = df_filtered.groupBy("country_name").sum("co2_emission_mt")

# Mostrar el total de emisiones de CO2 por país
total_co2_by_country.show(10)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+--------------------+
|country_name|sum(co2_emission_mt)|
+------------+--------------------+
|      Guyana|         2195725.491|
|      Turkey|     1.83696123205E8|
|Saint Helena|             6615.16|
|   Argentina|1.8167600238599998E8|
|      Angola|1.0040569527999999E7|
|     Albania|         6855301.317|
|   Nicaragua|  3342924.3349999995|
|        Peru|3.9960106194999985E7|
|       China|    3.571127691988E9|
|     Somalia|   738506.1030000001|
+------------+--------------------+
only showing top 10 rows

# 10. Guardar los Resultados en S3 en Formato Parquet

In [19]:
# Guardar los resultados en formato Parquet en el bucket de S3
total_co2_by_country.write.mode("overwrite").parquet("s3://tduquegtrabajo1-almac/trusted/country/co2_emissions_parquet")




FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…